In [32]:
# %%
import os
from dotenv import load_dotenv, find_dotenv
from getpass import getpass

# 1. Try to find and load .env file
env_path = find_dotenv()
if env_path:
    load_dotenv(env_path)
    print(f"Loaded environment variables from: {env_path}")
else:
    print("⚠️  No .env file found — falling back to manual input.")

# 2. Helper function to fetch or prompt
def get_env_var(key, prompt_text=None, secret=False):
    """Return environment variable or prompt user if missing."""
    value = os.getenv(key)
    if not value:
        if secret:
            value = getpass(prompt_text or f"Enter {key}: ")
        else:
            value = input(prompt_text or f"Enter {key}: ")
        os.environ[key] = value  # optionally keep it in memory
    return value

# 3. Use it
OPENAI_API_KEY = get_env_var("OPENAI_API_KEY", "OPENAI_API_KEY: ", secret=True)
LANGSMITH_API_KEY = get_env_var("LANGSMITH_API_KEY", "LANGSMITH_API_KEY: ", secret=True)
MCAP_FOLDER = os.getenv("MCAP_FOLDER", "/home/jackliu2006/workspace/mcap")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING", "true")
MILVUS_HOST=os.getenv("MILVUS_HOST", "localhost")
MILVUS_PORT=os.getenv("MILVUS_PORT", "19530")
MILVUS_DB=os.getenv("MILVUS_DB", "mbm")
COlLECTION_NAME=os.getenv("COLLECTION_NAME", "mcap")
# %%
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://genai-nexus.int.api.corpinter.net/apikey/",
    api_version="2024-06-01",
    api_key=OPENAI_API_KEY,
    model="text-embedding-3-large",
    dimensions=3072
)

# %%
from langchain_milvus import Milvus

from langchain_milvus import BM25BuiltInFunction, Milvus

URI = f"http://{MILVUS_HOST}:{MILVUS_PORT}"

vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI, "token": "root:Milvus", "db_name": MILVUS_DB},
    collection_name=COlLECTION_NAME,
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    consistency_level="Strong",
    drop_old=False,  # set to True if seeking to drop the collection with that name if it exists
    text_field="page_content",
    enable_dynamic_field=True,
    auto_id=True,
)

Loaded environment variables from: /Users/jackliu2006/workspace/hello-agent/.env


In [34]:
results = vectorstore.similarity_search(
    "What is mcap?",
    k=5,  
)
for res in results:
    # Try to print page_content if available, else print the whole object
    content = getattr(res, "page_content", None)
    if content is not None:
        print(f"* {content} [{res.metadata}]")
    else:
        print(f"* {res} [No page_content attribute]")

* #### Is mCAP for me?

You can find an exhaustive list of features and restrictions [here](./docs/scope.md).
This should help potential customers in their decision process about joining mCAP.

## Customer

The product teams in MBM are our customers and can become part of the swarm.

## Documentation

Documentation can be found in [`/docs`](/docs/) and all fundamental decisions are discussed and documented as [RFCs](/rfcs/).

## Guiding Principles [{'id': 461536897069481651, 'metadata': {'source': '/Users/jackliu2006/workspace/mcap/README.md'}}]
* # mCAP - MBM Cloud Azure Platform

<img src="docs/images/mCapwhite.png" alt="mCAP icon" width="200"/>

## Vision

Support and empower product teams to ship reliable stable products.
Reduction of overall complexity in tooling and infrastructure.
The platform can be used completely, partially or not at all.

## Goals [{'id': 461536897069481647, 'metadata': {'source': '/Users/jackliu2006/workspace/mcap/README.md'}}]
* # Contributing to mCAP

mCA